In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rpy2.robjects as ro
from functools import partial
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.ipython import html
html.html_rdataframe = partial(html.html_rdataframe, table_class = "docutils")
%load_ext rpy2.ipython

In [2]:
%%R
library(mlbench)
datasetxor <- mlbench.xor(100)
XR_datasetxor <- datasetxor$x
LABELSR_datasetxor<- datasetxor$classes

In addition: Warning message:
In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/usr/lib/R/site-library’ contains no packages


In [3]:
from rpy2.robjects import numpy2ri
numpy2ri.activate()
x_df_xor = np.array(ro.r['XR_datasetxor'])
labels_df_xor = np.array(ro.r['LABELSR_datasetxor'])
labels_df_xor[labels_df_xor == 1] = -1
labels_df_xor[labels_df_xor == 2] = 1

In [4]:
from sklearn.model_selection import train_test_split
X_trainxor, X_testxor, y_trainxor, y_testxor = train_test_split(x_df_xor, labels_df_xor, random_state = 0, train_size = 0.7)

In [5]:
def train_ELM(xin : np.ndarray, yin : np.ndarray, p : int, control : bool, lam : float) -> list:
    n = xin.shape[1]
    
    if control == True:
        Z = np.array([np.random.uniform(-0.5, 0.5) for _ in range((n + 1) * p)]).reshape(n + 1, -1)
        ones = np.ones((xin.shape[0], 1))
        xin = np.concatenate((xin,ones), axis = 1)
    else:
        Z = np.array([np.random.uniform(-0.5, 0.5) for _ in range(n * p)]).reshape(n , -1)

    H = np.tanh(np.dot(xin, Z))
    ones = np.ones((H.shape[0], 1))
    H = np.concatenate((H, ones), axis = 1)

    diagonal_matrix = lam * np.eye(H.shape[1])
    # w = (HT H + λIp)^(-1)HT y
    w1 = np.linalg.inv(np.dot(np.transpose(H), H) + diagonal_matrix)
    w1 = np.linalg.inv(np.dot(np.transpose(H), H) + diagonal_matrix)
    w = np.dot(w1, np.dot(np.transpose(H), yin))

    return_list = []
    return_list.append(w)   
    return_list.append(H)
    return_list.append(Z)
    return  return_list



def test_ELM(xin: np.ndarray, Z: np.ndarray, W: np.ndarray, control: bool):
    
    if control == True:
        ones = np.ones((xin.shape[0], 1))
        xin = np.concatenate((xin, ones), axis = 1)
    H = np.tanh(np.dot(xin, Z))
    ones = np.ones((H.shape[0], 1))
    H = np.concatenate((H, ones), axis = 1)
    Y_hat = np.sign(np.dot(H, W))
    return Y_hat

In [6]:
from sklearn.model_selection import cross_validate
pxor = 100
lst_errors_lam = list()
lst_pesos_w = list()
lam = 0
stop = 5
lst_lambda = list()
#wxor_aux = 0
#lst_wxor = list()
while lam <= stop:
    
    train_ELMspirals = train_ELM(X_trainxor, y_trainxor, pxor, control = True, lam = lam)
    wxor = np.array(train_ELMspirals[0])
    #if lam == 0:
    #   wxor = np.full(wxor.shape, -0.07916484416812761)
    Hxor = np.array(train_ELMspirals[1])
    Zxor = np.array(train_ELMspirals[2])
    y_hatrainxor = test_ELM(X_trainxor, Zxor, wxor, True)
    y_hatestxor = test_ELM(X_testxor, Zxor, wxor, True)

    # Pegando os erros de treino e os erros de teste.
    #sum_result = 0
    #for i in y_hatestxor:
     #   sum_ = (y_testxor - i)**2
     #   sum_result += sum_
    #sum_result = sum_result / y_testxor.shape[0]
    scores = cross_validate(train_ELMspirals, x_df_xor, labels_df_xor, cv = 5, scoring = 'accuracy')


    lst_errors_lam.append(sum_result)
    #lst_wxor.append(wxor)
    lst_lambda.append(lam)
    lst_pesos_w.append(np.linalg.norm(wxor)) # Norma do vetor w.
    lam = lam + 0.4
lst_errors_lam = np.array(lst_errors_lam)
lst_pesos_w = np.array(lst_pesos_w)
lst_lambda = np.array(lst_lambda)

InvalidParameterError: The 'estimator' parameter of cross_validate must be an object implementing 'fit'. Got [array([ 1.12387077e+06,  4.26969030e+07,  3.93637562e+07, -1.59918679e+08,
        2.56041289e+07, -8.20723927e+06, -2.92677775e+07,  2.02782121e+06,
        1.30695360e+07,  2.67210329e+07, -8.57417212e+06,  5.44982226e+06,
       -6.21949538e+06, -1.48929671e+07, -2.10543105e+07,  5.55952324e+05,
        4.18174702e+06, -8.76031336e+05, -7.38072463e+06,  1.48487277e+07,
       -7.29278723e+07,  6.46570182e+06, -2.31072442e+07,  1.01178042e+06,
        6.02019667e+06, -2.14566998e+07,  5.69425398e+06, -2.22457968e+07,
       -2.84075469e+07, -2.69824231e+07,  1.93874095e+06,  1.92650678e+07,
        3.93206013e+07, -1.89620786e+07,  1.42529846e+07, -3.31894686e+07,
        4.04479338e+06, -4.04686842e+07,  2.15869935e+06,  9.47527845e+06,
        1.98627331e+07, -6.59611311e+05, -2.44694704e+07,  5.19168598e+05,
        2.01367274e+06,  1.75363182e+07,  2.18969177e+07, -8.3183

In [ ]:
lst_errors_lam.min()
lst_errors_lam[lst_errors_lam == 1.4666666666666666]

In [ ]:
# Transformando as listas em arrays numpy.
lst_errors_lam = np.array(lst_errors_lam)
lst_lambda = np.array(lst_lambda)
lst_pesos_w = np.array(lst_pesos_w)
plt.scatter(lst_pesos_w, lst_errors_lam, color = 'black', label = 'lst_errors_lam x lst_pesos_w')
plt.title('lst_errors_lam x lst_pesos_w')
plt.xlabel('lst_pesos_w')
plt.xlim(0, 15)
plt.ylim(0, 100)
plt.ylabel('lst_errors_lam')
plt.legend()
plt.plot()

In [ ]:
print(np.max(lst_pesos_w))

In [ ]:
lst_pesos_w

In [ ]:
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(y_trainxor, y_hatrainxor)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
cm_display.plot()
plt.show()
sum_train = sum(y_trainxor != y_hatrainxor)
print(f"There are a total of {sum_train} errors in the train.")

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_testxor, y_hatestxor)
cm_display_ = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
cm_display_.plot()
plt.show()
sum_test = sum(y_testxor != y_hatestxor)
print(f"There are a total of {sum_test} errors in the test.")

In [ ]:
labels_df_reshaped = labels_df_xor.reshape(-1, 1)
mat_plot = np.concatenate((x_df_xor, labels_df_reshaped), axis = 1)
index_sort = 2
sorted_indices = np.argsort(mat_plot[:, index_sort])
mat_plot = mat_plot[sorted_indices]

In [ ]:
np.unique(mat_plot[:, 2], return_counts = True)

In [ ]:
x1_points = mat_plot[: 46, 0 : 2]
x2_points = mat_plot[46 :, 0 : 2]
plt.scatter(x1_points[:, 0], x1_points[:, 1], color = 'red', label = 'data 1')
plt.scatter(x2_points[:, 0], x2_points[:, 1], color = 'blue', label = 'data 2')
plt.title('data')
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.plot()

In [ ]:
seqx1x2 = np.linspace(start = -4, stop = 4, num = 300)
np_grid = seqx1x2.shape[0]
shape = (np_grid, np_grid)
MZ = np.zeros(shape)
for i in range(np_grid):
    for j in range(np_grid):
        x1 = seqx1x2[i]
        x2 = seqx1x2[j]
        x1x2 = np.column_stack((x1, x2, 1))
        h1 = np.tanh(np.dot(x1x2, Zxor))
        h1 = np.column_stack((h1, np.ones_like(h1[:, 0])))
        MZ[i, j] = np.sign(np.dot(h1, wxor))[0]


plt.contour(seqx1x2, seqx1x2, MZ, levels = 1)
plt.scatter(x1_points[:, 0], x1_points[:, 1], color = 'red', label = 'data1')
plt.scatter(x2_points[:, 0], x2_points[:, 1], color = 'blue', label = 'data2')
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Contour Plot')
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.grid(True)
plt.show()